In [26]:
from bs4 import BeautifulSoup, NavigableString, Tag

In [27]:
import pandas as pd
import re
import requests
import numpy as np

In [28]:
base_url = "https://catalogs.gsu.edu/content.php?catoid=5&navoid=606"

In [29]:
def extraction():
    data =[]
    data_1 =[]
    data_2 =[]
    data_3 =[]
    dept =[]
    department =[]
    for r in range(1,39):
        query_parameter = "&filter%5Bitem_type%5D=3&filter%5Bonly_active%5D=1&filter%5B3%5D=1&filter%5Bcpage%5D="+str(r)+"#acalog_template_course_filter"
        url = base_url + query_parameter
        result = requests.get(url)
        #print(result)
        #doc = BeautifulSoup(result.text,"html.parser")
        #description = doc.find_all("td",attrs ={"class","width"})
        #print(description)
        #for i in range(len(description)):
                #q =(description[i].find("a").text.replace("\xa0"," "))
                #data0 = q.split("-",1)
                #data.append(data0[0])
               # data_1.append(data0[1])
        #desc = doc.find_all("td", class_ = 'block_content')
        #for ii in range(len(desc)):
                #d = (desc[ii].find("strong").text.replace("\xa0"," "))
                #data_3.append(d)
                
        #print(len(data))
        #print(len(data_1))
        #print(len(data_3))
        doc1 = BeautifulSoup(result.text,"html.parser")
        description1 = doc1.find('td', class_ ='block_content')
        #print(description1)
        list_links =[]
        dict_href_links={}
        for d1 in description1.find_all("strong"):
            d2 = d1.text
            if(d2 != 'HELP' and d2!= str(r)):
                department.append(d2.strip())
        #print(department)
        for link in description1.find_all("a", href=True):
            if str(link["href"]).startswith("preview"):
                #print(link['href'])
                dict_href_links[link["href"]] = None
                link_with_www = 'http://catalogs.gsu.edu/' + link["href"]
            # print("adjusted link =", link_with_www)
                list_links.append(link_with_www)
        #for ll in list_links:
            #print(ll)
        #print(len(list_links))
        for i in range(len(list_links)):
            url1 = list_links[i]
            html_text = requests.get(url1).text
            soup1 = BeautifulSoup(html_text, 'html.parser')
            desc_page = soup1.find('td', class_='block_content')
            #print(desc_page)
            course = desc_page.find('h1').text.split('-', 1)
            #course_desc = desc_page.find('hr').find_next_sibling(text=True)
            course_id = course[0]
            data.append(course_id.strip())
            dpt = course_id.split(" ",1)
            dept.append(dpt[0].strip())
            course_title = course[1]
            data_1.append(course_title.strip())
            #for d in range(len(desc_page)):
            #print(desc_page.text.split('\n'))
            data_desc = desc_page.text.split('\n')[22]
            data_desc1 = data_desc.split('-',1)
            if (data_desc1):
                data_2.append(data_desc1[1].strip())
            else:
                data_2.append('No description for this course')
        #print(dept)
        #print(len(data))
        #print(len(data_1))
        #print(len(data_2))
        #print(len(course_id))
        #print(len(course_title))
        
    return data,data_1,data_2,department,dept
courses = extraction()

In [32]:
courses_df = pd.DataFrame({'CourseNumber':courses[0], 'CourseTitle':courses[1], 'Description':courses[2], 'CourseDepartment':courses[4]})
courses_df.to_csv('../data/georgia_graduate_temp.csv')

In [33]:
courses_df

,CourseNumber,CourseTitle,Description,CourseDepartment
0,ACCT 8005,Current Issues in Accounting,Current Issues in Accounting1 to 3 Credit Hour...,ACCT
1,ACCT 8010,Financial Accounting 1,Financial Accounting 13 Credit Hours Prerequis...,ACCT
2,ACCT 8020,Financial Accounting II,Financial Accounting II3 Credit Hours Prerequi...,ACCT
3,ACCT 8030,Managerial and Accounting Information Systems,Managerial and Accounting Information Systems3...,ACCT
4,ACCT 8040,Topics in Federal Taxation,Topics in Federal Taxation3 Credit Hours Prere...,ACCT
...,...,...,...,...
3720,WLC 8320,Cultural and Literary Analysis and Production,Cultural and Literary Analysis and Production3...,WLC
3721,WLC 8330,"The Media, the Arts, and Popular Culture","The Media, the Arts, and Popular Culture3 Cred...",WLC
3722,WLC 8340,Learning a World Language,Learning a World Language3 Credit Hours Descr...,WLC
3723,WLC 8350,"Identities, Borders, and Social Justice in Lan...","Identities, Borders, and Social Justice in Lan...",WLC


In [34]:
dept1 =courses[3]
dept1 = np.unique(dept1)
print(len(dept1))
print(dept1)

114
['Accounting' 'Actuarial Science' 'Africana Studies' 'Analytics'
 'Anthropology' 'Applied Linguistics' 'Applied Organ' 'Art'
 'Art Education' 'Art History' 'Astronomy' 'Biology' 'Biomedical Sciences'
 'Birth Through Five' 'Business Administration'
 'Business Administration - BUSA' 'Business Communication'
 'Center Processing Innovation' 'Ceramics' 'Chemistry'
 'College of Nursing & Health Professions' 'Communication'
 'Communication & Science Disorders' 'Computer Information Systems'
 'Computer Science' 'Counseling & Psychological Services'
 'Criminal Justice' 'Doctorate in Business Administration'
 'Drawing and Painting' 'EPS/Educational Leadership'
 'EPS/Higher Education' 'EPS/Research' 'EPS/Social Foundations'
 'Early Childhood Education' 'Economics' 'Education'
 'Education/Curriculum and Instruction' 'Educational Policy Studies'
 'Educational Psychology' 'English' 'English as a Second Language'
 'Entrepreneurship Minor' 'Exceptional Children'
 'Executive Doctorate in Business'


In [35]:
x1 = (courses_df.CourseDepartment.value_counts(sort=False))
xi2_1 = list(x1)
print(len(xi2_1))
xi2_1

114


[26,
 9,
 30,
 19,
 67,
 54,
 1,
 10,
 17,
 32,
 24,
 122,
 22,
 3,
 8,
 2,
 2,
 3,
 9,
 61,
 49,
 37,
 84,
 40,
 104,
 82,
 40,
 7,
 16,
 101,
 72,
 3,
 25,
 14,
 35,
 83,
 5,
 11,
 21,
 6,
 37,
 22,
 17,
 59,
 27,
 61,
 29,
 5,
 17,
 33,
 79,
 6,
 26,
 23,
 23,
 19,
 19,
 73,
 11,
 4,
 21,
 11,
 32,
 85,
 8,
 2,
 46,
 44,
 11,
 20,
 14,
 73,
 12,
 7,
 1,
 117,
 3,
 85,
 97,
 54,
 54,
 2,
 62,
 13,
 67,
 53,
 69,
 6,
 13,
 71,
 64,
 55,
 69,
 5,
 14,
 16,
 57,
 38,
 14,
 1,
 9,
 10,
 55,
 8,
 35,
 34,
 29,
 6,
 18,
 3,
 9,
 20,
 49,
 8]

In [36]:
lenlist1=[]
for i in range(len(dept1)):
    xi = int(xi2_1[i])
    while(xi!=0):
        lenlist1.append(dept1[i])
        xi = xi-1
        
lenlist1
p1 = pd.DataFrame(lenlist1)

In [37]:
p1

,0
0,Accounting
1,Accounting
2,Accounting
3,Accounting
4,Accounting
...,...
3720,World Languages and Cultures
3721,World Languages and Cultures
3722,World Languages and Cultures
3723,World Languages and Cultures


In [38]:
courses_df['Department'] = lenlist1

In [39]:
courses_df

,CourseNumber,CourseTitle,Description,CourseDepartment,Department
0,ACCT 8005,Current Issues in Accounting,Current Issues in Accounting1 to 3 Credit Hour...,ACCT,Accounting
1,ACCT 8010,Financial Accounting 1,Financial Accounting 13 Credit Hours Prerequis...,ACCT,Accounting
2,ACCT 8020,Financial Accounting II,Financial Accounting II3 Credit Hours Prerequi...,ACCT,Accounting
3,ACCT 8030,Managerial and Accounting Information Systems,Managerial and Accounting Information Systems3...,ACCT,Accounting
4,ACCT 8040,Topics in Federal Taxation,Topics in Federal Taxation3 Credit Hours Prere...,ACCT,Accounting
...,...,...,...,...,...
3720,WLC 8320,Cultural and Literary Analysis and Production,Cultural and Literary Analysis and Production3...,WLC,World Languages and Cultures
3721,WLC 8330,"The Media, the Arts, and Popular Culture","The Media, the Arts, and Popular Culture3 Cred...",WLC,World Languages and Cultures
3722,WLC 8340,Learning a World Language,Learning a World Language3 Credit Hours Descr...,WLC,World Languages and Cultures
3723,WLC 8350,"Identities, Borders, and Social Justice in Lan...","Identities, Borders, and Social Justice in Lan...",WLC,World Languages and Cultures


In [40]:
courses_df = courses_df.drop(['CourseDepartment'],axis =1)
courses_df

,CourseNumber,CourseTitle,Description,Department
0,ACCT 8005,Current Issues in Accounting,Current Issues in Accounting1 to 3 Credit Hour...,Accounting
1,ACCT 8010,Financial Accounting 1,Financial Accounting 13 Credit Hours Prerequis...,Accounting
2,ACCT 8020,Financial Accounting II,Financial Accounting II3 Credit Hours Prerequi...,Accounting
3,ACCT 8030,Managerial and Accounting Information Systems,Managerial and Accounting Information Systems3...,Accounting
4,ACCT 8040,Topics in Federal Taxation,Topics in Federal Taxation3 Credit Hours Prere...,Accounting
...,...,...,...,...
3720,WLC 8320,Cultural and Literary Analysis and Production,Cultural and Literary Analysis and Production3...,World Languages and Cultures
3721,WLC 8330,"The Media, the Arts, and Popular Culture","The Media, the Arts, and Popular Culture3 Cred...",World Languages and Cultures
3722,WLC 8340,Learning a World Language,Learning a World Language3 Credit Hours Descr...,World Languages and Cultures
3723,WLC 8350,"Identities, Borders, and Social Justice in Lan...","Identities, Borders, and Social Justice in Lan...",World Languages and Cultures


In [41]:
courses_df.rename(columns={'Description':'CourseDescription','Department':'CourseDepartment'}, inplace = True)

In [42]:
courses_df

,CourseNumber,CourseTitle,CourseDescription,CourseDepartment
0,ACCT 8005,Current Issues in Accounting,Current Issues in Accounting1 to 3 Credit Hour...,Accounting
1,ACCT 8010,Financial Accounting 1,Financial Accounting 13 Credit Hours Prerequis...,Accounting
2,ACCT 8020,Financial Accounting II,Financial Accounting II3 Credit Hours Prerequi...,Accounting
3,ACCT 8030,Managerial and Accounting Information Systems,Managerial and Accounting Information Systems3...,Accounting
4,ACCT 8040,Topics in Federal Taxation,Topics in Federal Taxation3 Credit Hours Prere...,Accounting
...,...,...,...,...
3720,WLC 8320,Cultural and Literary Analysis and Production,Cultural and Literary Analysis and Production3...,World Languages and Cultures
3721,WLC 8330,"The Media, the Arts, and Popular Culture","The Media, the Arts, and Popular Culture3 Cred...",World Languages and Cultures
3722,WLC 8340,Learning a World Language,Learning a World Language3 Credit Hours Descr...,World Languages and Cultures
3723,WLC 8350,"Identities, Borders, and Social Justice in Lan...","Identities, Borders, and Social Justice in Lan...",World Languages and Cultures


In [44]:
courses_df.to_csv('../data/georgia_grad_final.csv', index = False)

In [45]:
courses_df.isnull().sum()

CourseNumber         0
CourseTitle          0
CourseDescription    0
CourseDepartment     0
dtype: int64

In [47]:
grad_df = pd.read_csv('../data/georgia_grad_final.csv')
grad_df

,CourseNumber,CourseTitle,CourseDescription,CourseDepartment
0,ACCT 8005,Current Issues in Accounting,Current Issues in Accounting1 to 3 Credit Hour...,Accounting
1,ACCT 8010,Financial Accounting 1,Financial Accounting 13 Credit Hours Prerequis...,Accounting
2,ACCT 8020,Financial Accounting II,Financial Accounting II3 Credit Hours Prerequi...,Accounting
3,ACCT 8030,Managerial and Accounting Information Systems,Managerial and Accounting Information Systems3...,Accounting
4,ACCT 8040,Topics in Federal Taxation,Topics in Federal Taxation3 Credit Hours Prere...,Accounting
...,...,...,...,...
3720,WLC 8320,Cultural and Literary Analysis and Production,Cultural and Literary Analysis and Production3...,World Languages and Cultures
3721,WLC 8330,"The Media, the Arts, and Popular Culture","The Media, the Arts, and Popular Culture3 Cred...",World Languages and Cultures
3722,WLC 8340,Learning a World Language,Learning a World Language3 Credit Hours Descr...,World Languages and Cultures
3723,WLC 8350,"Identities, Borders, and Social Justice in Lan...","Identities, Borders, and Social Justice in Lan...",World Languages and Cultures


In [25]:
read_file = pd.read_csv ('../data/georgia_grad_final.csv', sep = '\t')
read_file.to_excel ('../data/georgia_grad_final.xlsx', index = None, header=True)